In [1]:
import numpy as np
import os as os
import torch
import torch.nn as nn
import cv2
from PIL import Image

######
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
# Define the model architecture
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = x.view(-1, 128 * 8 * 8)
        x = nn.functional.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x
    
model = CNN()

input_tensor = torch.randn(1, 3, 70, 70)

output = model.forward(input_tensor)
print(np.array(input_tensor).shape)

(1, 3, 70, 70)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


In [3]:
def create_images(img_folder, new_folder):
    
    for dir1 in os.listdir(img_folder):
        
        if dir1 != '.DS_Store':
            
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                new_name = os.path.splitext(file)
                
                
                image_path = os.path.join(img_folder, dir1)
                new_image_path = os.path.join(new_folder, dir1)

                im1 = Image.open('{}/{}'.format(image_path, file))

                rbg_im1 = im1.convert('RGB')


                rbg_im1.save('{}/{}.jpg'.format(new_image_path, new_name[0]))
                
    return 0

# create_images('fixed_images', 'new_images')

In [4]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for dir1 in os.listdir(img_folder):
        if dir1 != '.DS_Store':
            for file in os.listdir(os.path.join(img_folder, dir1)):
                
                
       
                image_path= os.path.join(img_folder, dir1,  file)
                image= cv2.imread( image_path, 1)
#                 image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image, (100, 100),interpolation = cv2.INTER_AREA)
                image=np.array(image)
                image = image.astype('float32')
                image /= 255 
                img_data_array.append(image)
                class_name.append(dir1)
                
    return img_data_array, class_name# extract the image array and class name
'''  temp  |  lytic   '''

dataset_X, dataset_y = create_dataset('new_images')


In [5]:
dataset_X = np.array(dataset_X)
dataset_y = np.array(dataset_y)

In [6]:
# img_folder = 'fixed_images'
# img_data_array, class_name = create_dataset(img_folder)

# label_dict = {'temperate': 0, 'lytic': 1}
# class_labels = [label_dict[label] for label in class_name]

# class_labels_one_hot = to_categorical(class_labels)



# X_train, X_test, y_train, y_test = train_test_split(img_data_array, class_labels_one_hot, test_size=0.2, random_state=42)

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(2, activation='softmax')
# ])

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.fit(np.array(X_train), np.array(y_train), batch_size=22, epochs=100, verbose=0, validation_split=0.2)

# loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
# print(f"Test accuracy: {accuracy}")

# model.save('temperate_lytic_classifier.h5')


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.image import rgb_to_grayscale


img_folder = 'fixed_images'
img_data_array, class_name = create_dataset(img_folder)

label_dict = {'temperate': 0, 'lytic': 1}
class_labels = [label_dict[label] for label in class_name]

new_y = to_categorical(class_labels)

X_train, X_test, y_train, y_test = train_test_split(img_data_array, new_y, test_size=0.2, random_state=42)

# This code might break it, so remove in the future

X_train = rgb_to_grayscale(X_train)
X_test = rgb_to_grayscale(X_test)

############################

def kernelInitializer(shape, dtype=None):   
    sobel_x = tf.constant(
        [
            [-5, -4, 0, 4, 5], 
            [-8, -10, 0, 10, 8], 
            [-10, -20, 0, 20, 10], 
            [-8, -10, 0, 10, 8], 
            [-5, -4, 0, 4, 5]
        ], dtype=dtype )
    
    sobel_x = tf.reshape(sobel_x, (5, 5, 1, 1))
    
    sobel_x = tf.tile(sobel_x, (1, 1, shape[-2],shape[-1]))

    print(tf.shape(sobel_x))
    
    return sobel_x

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Model with more layers, batch normalization, and L1/L2 regularization
model = Sequential([
    Conv2D(64, (5, 5), activation='relu', input_shape=(100, 100, 1), strides=(2,2)),
#     BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu', strides=(2,2)),
#     BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(16, (3, 3), activation='relu', strides=(2,2)),
#     BatchNormalization(),
#     MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    
#     Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4)),
#     Dense(232, activation='relu'),
    Dense(216, activation='relu'),
    Dense(200, activation='relu'),
    Dense(180, activation='relu'),
    Dense(150, activation='relu'),
    Dense(125, activation='relu'),
    Dense(90, activation='relu'),
    Dense(75, activation='relu'),
    Dense(50, activation='relu'),
    Dense(25, activation='relu'),
    
#     Dropout(0.2),
    
    Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# Early stopping
# early_stopping = EarlyStopping(
#     monitor='val_loss', patience=10, restore_best_weights=True)

training_set = datagen.flow(np.array(X_train), np.array(y_train), batch_size=25)

# Fit the model with data augmentation

# Can use 'training_set' for overfitting, or can use X_train and y_train straight up
model.fit(X_train,
          y_train,
          batch_size=20,
          epochs=100,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test))
          #
         )

print('Model before using DataGenerator:\n\n')
loss, accuracy = model.evaluate(np.array(X_train), np.array(y_train), verbose=1)
print(f"Training accuracy: {accuracy}\n")
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}\n\n")

model.fit(training_set,
          steps_per_epoch=np.array(X_train).shape[0] // 40,
          epochs=100,
          verbose=1,
          validation_data=(np.array(X_test), np.array(y_test))
          #
         )

print('Model with after using DataGenerator:\n\n')
loss, accuracy = model.evaluate(np.array(X_train), np.array(y_train), verbose=1)
print(f"Training accuracy: {accuracy}\n")
loss, accuracy = model.evaluate(np.array(X_test), np.array(y_test), verbose=1)
print(f"Test accuracy: {accuracy}")

model.save('temperate_lytic_classifier_v2.h5')


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: sRGB: gamma value does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


Epoch 1/100
39/39 [==============================] - 1s 16ms/step - loss: 0.6874 - accuracy: 0.5681 - val_loss: 0.6792 - val_accuracy: 0.5846
Epoch 2/100
39/39 [==============================] - 0s 12ms/step - loss: 0.6870 - accuracy: 0.5707 - val_loss: 0.6810 - val_accuracy: 0.5846
Epoch 3/100
39/39 [==============================] - 0s 12ms/step - loss: 0.6836 - accuracy: 0.5707 - val_loss: 0.6886 - val_accuracy: 0.5846
Epoch 4/100
39/39 [==============================] - 0s 12ms/step - loss: 0.6742 - accuracy: 0.5707 - val_loss: 0.6810 - val_accuracy: 0.5846
Epoch 5/100
39/39 [==============================] - 0s 12ms/step - loss: 0.6879 - accuracy: 0.5707 - val_loss: 0.6824 - val_accuracy: 0.5846
Epoch 6/100
39/39 [==============================] - 0s 12ms/step - loss: 0.6853 - accuracy: 0.5707 - val_loss: 0.6802 - val_accuracy: 0.5846
Epoch 7/100
39/39 [==============================] - 0s 13ms/step - loss: 0.6845 - accuracy: 0.5707 - val_loss: 0.6799 - val_accuracy: 0.5846
Epoch 

39/39 [==============================] - 0s 12ms/step - loss: 0.6039 - accuracy: 0.6350 - val_loss: 0.8729 - val_accuracy: 0.6103
Epoch 59/100
39/39 [==============================] - 0s 11ms/step - loss: 0.5326 - accuracy: 0.7301 - val_loss: 0.8046 - val_accuracy: 0.6872
Epoch 60/100
39/39 [==============================] - 0s 11ms/step - loss: 0.4574 - accuracy: 0.7828 - val_loss: 0.5380 - val_accuracy: 0.7795
Epoch 61/100
39/39 [==============================] - 0s 11ms/step - loss: 0.4583 - accuracy: 0.7853 - val_loss: 0.7091 - val_accuracy: 0.7282
Epoch 62/100
39/39 [==============================] - 0s 13ms/step - loss: 0.6144 - accuracy: 0.6350 - val_loss: 0.7276 - val_accuracy: 0.6103
Epoch 63/100
39/39 [==============================] - 0s 12ms/step - loss: 0.5822 - accuracy: 0.6748 - val_loss: 0.8100 - val_accuracy: 0.4667
Epoch 64/100
39/39 [==============================] - 0s 12ms/step - loss: 0.5184 - accuracy: 0.7249 - val_loss: 0.9137 - val_accuracy: 0.6256
Epoch 65/100

19/19 [==============================] - 0s 23ms/step - loss: 0.4800 - accuracy: 0.7925 - val_loss: 0.4249 - val_accuracy: 0.8051
Epoch 14/100
19/19 [==============================] - 0s 23ms/step - loss: 0.4305 - accuracy: 0.7979 - val_loss: 0.4187 - val_accuracy: 0.7949
Epoch 15/100
19/19 [==============================] - 0s 25ms/step - loss: 0.4366 - accuracy: 0.8057 - val_loss: 0.4143 - val_accuracy: 0.7949
Epoch 16/100
19/19 [==============================] - 0s 23ms/step - loss: 0.4638 - accuracy: 0.8057 - val_loss: 0.4752 - val_accuracy: 0.7846
Epoch 17/100
19/19 [==============================] - 0s 24ms/step - loss: 0.4299 - accuracy: 0.8124 - val_loss: 0.3991 - val_accuracy: 0.8154
Epoch 18/100
19/19 [==============================] - 0s 23ms/step - loss: 0.4808 - accuracy: 0.7853 - val_loss: 0.4204 - val_accuracy: 0.8205
Epoch 19/100
19/19 [==============================] - 0s 23ms/step - loss: 0.4432 - accuracy: 0.7937 - val_loss: 0.4243 - val_accuracy: 0.7795
Epoch 20/100

19/19 [==============================] - 0s 23ms/step - loss: 0.4184 - accuracy: 0.8105 - val_loss: 0.4025 - val_accuracy: 0.8359
Epoch 71/100
19/19 [==============================] - 0s 25ms/step - loss: 0.3664 - accuracy: 0.8611 - val_loss: 0.4110 - val_accuracy: 0.8308
Epoch 72/100
19/19 [==============================] - 0s 24ms/step - loss: 0.3915 - accuracy: 0.8344 - val_loss: 0.3728 - val_accuracy: 0.8205
Epoch 73/100
19/19 [==============================] - 0s 23ms/step - loss: 0.3918 - accuracy: 0.8316 - val_loss: 0.3603 - val_accuracy: 0.8462
Epoch 74/100
19/19 [==============================] - 0s 24ms/step - loss: 0.3466 - accuracy: 0.8526 - val_loss: 0.3520 - val_accuracy: 0.8667
Epoch 75/100
19/19 [==============================] - 0s 23ms/step - loss: 0.3632 - accuracy: 0.8484 - val_loss: 0.3730 - val_accuracy: 0.8410
Epoch 76/100
19/19 [==============================] - 0s 23ms/step - loss: 0.3500 - accuracy: 0.8653 - val_loss: 0.4166 - val_accuracy: 0.8410
Epoch 77/100

In [8]:


shape = dataset_X.shape

print(dataset_X.shape)

dataset_X = np.reshape(dataset_X, (shape[0],shape[3],shape[1],shape[2]))

print(dataset_X.shape)

output = model.forward(torch.tensor(dataset_X))

print()

(973, 100, 100, 3)
(973, 3, 100, 100)


AttributeError: 'Sequential' object has no attribute 'forward'

In [ ]:
torch.compile(CNN())

In [ ]:
# DO NOT RUN THIS IF YOU WANT TO KEEP KERNEL ALIVE

import tensorflow as tf
from tensorflow.keras import layers

# Define the model architecture
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])